Run in enviorment: `source 2020-2021/Spring2021/greco_grb/gbm/bin/activate` 

In [1]:
import numpy as np

In [2]:
from gbm.finder import BurstCatalog

ModuleNotFoundError: No module named 'gbm'

In [5]:
burstcat = BurstCatalog()
burstcat.num_rows

Finished in 70 s


2973

In [6]:
burstcat.columns

array(['name', 'ra', 'dec', 'trigger_time', 't90', 't90_error',
       't90_start', 'fluence', 'fluence_error', 'flux_1024',
       'flux_1024_error', 'flux_1024_time', 'flux_64', 'flux_64_error',
       'flnc_band_ampl', 'flnc_band_ampl_pos_err',
       'flnc_band_ampl_neg_err', 'flnc_band_epeak',
       'flnc_band_epeak_pos_err', 'flnc_band_epeak_neg_err',
       'flnc_band_alpha', 'flnc_band_alpha_pos_err',
       'flnc_band_alpha_neg_err', 'flnc_band_beta',
       'flnc_band_beta_pos_err', 'flnc_band_beta_neg_err',
       'flnc_spectrum_start', 'flnc_spectrum_stop',
       'pflx_best_fitting_model', 'pflx_best_model_redchisq',
       'flnc_best_fitting_model', 'flnc_best_model_redchisq',
       'actual_1024ms_interval', 'actual_256ms_interval',
       'actual_64ms_interval', 'back_interval_high_start',
       'back_interval_high_stop', 'back_interval_low_start',
       'back_interval_low_stop', 'bcat_detector_mask', 'bcatalog', 'bii',
       'duration_energy_high', 'duration_energy

In [10]:
import pandas

In [11]:
df = pandas.DataFrame(burstcat.get_table())
df.head()

,name,ra,dec,trigger_time,t90,t90_error,t90_start,fluence,fluence_error,flux_1024,...,pflx_sbpl_redfitstat,pflx_sbpl_statistic,pflx_spectrum_start,pflx_spectrum_stop,scat_detector_mask,scatalog,t50,t50_error,t50_start,trigger_name
0,GRB120403857,55.338333,-89.009167,2012-04-03 20:33:58.493,4.288,1.935,-3.968,2.396400e-07,2.045800e-08,1.7469,...,1.156,Castor C-STAT,-1.024,0.000,11001,3,1.408,1.620,-1.536,bn120403857
1,GRB120227725,256.730000,-88.860000,2012-02-27 17:24:41.054,17.408,0.810,0.256,2.194900e-05,1.040300e-07,18.4653,...,1.218,Castor C-STAT,7.168,8.192,100101,3,6.656,0.572,5.632,bn120227725
2,GRB141205018,294.610000,-87.580000,2014-12-05 00:25:29.813,13.056,1.280,-6.912,1.603100e-06,4.126700e-08,2.1505,...,1.201,Castor C-STAT,5.120,6.144,1000100001,3,5.888,1.280,-1.792,bn141205018
3,GRB180630467,48.958750,-87.478611,2018-06-30 11:11:54.405,12.032,1.864,-2.048,2.988000e-06,3.253700e-08,9.4128,...,1.070,Castor C-STAT,1.024,2.048,11010001,3,2.816,0.362,0.768,bn180630467
4,GRB170116238,72.940000,-87.430000,2017-01-16 05:43:15.259,9.216,3.620,-1.024,1.123100e-06,9.197500e-08,5.4422,...,1.110,Castor C-STAT,-1.024,0.000,100100000010,3,4.352,2.111,-0.256,bn170116238


Download healpix map if it exists \
otherwise make a fake one

In [13]:
grbweb = pandas.read_pickle("/home/cjchen/2020-2021/Spring2021/greco_grb/data/grbweb/grb_grbweb.pkl")
grbweb.head()

,grb_name,grb_name_GBM,ra,dec,mjd_start,t100,t90,gbm,source_uncertainty,redshift,t_center
0,GRB190613B,GRB190613449,305.431917,-4.646667,58647.449285,0.001905,0.000056,0,0.000056,NaN,58647.450238
1,GRB190613A,GRB190613172,182.529208,67.235278,58647.171736,0.000204,0.000198,0,0.000056,NaN,58647.171838
2,GRB190612A,GRB190612165,223.950000,62.100000,58646.164836,0.001677,0.001677,1,7.025096,NaN,58646.165675
3,GRB190611B,GRB190611950,84.758333,50.833333,58645.949833,0.001217,0.001164,0,5.900000,NaN,58645.950442
4,GRB190611A,None,324.722625,-56.074278,58645.742373,0.000484,0.000484,0,0.000194,NaN,58645.742615


In [43]:
import os
import glob
from gbm.finder import TriggerFtp

In [29]:
trig_finder = TriggerFtp()
trig_finder.num_files

0

In [53]:
# name_set = set(df.name)
# download_folder = "/home/cjchen/2020-2021/Spring2021/greco_grb/data/gbm/healpix_ftp"   # a softlinked folder
# for name in grbweb.grb_name_GBM:
#     if name in name_set:
#         trig_finder.set_trigger(name[3:])
#         if any("healpix" in filename for filename in trig_finder.ls_localization()):
#             name_set.remove(name)    # it has a healpix map
#             trig_finder.get_localization(download_folder)
#             all_files = glob.glob('{}/*'.format(download_folder))
#             healpix_files = glob.glob('{}/*_healpix_*.fit'.format(download_folder))
#             res = list(set(all_files) - set(healpix_files))
#             for f in res:
#                 try:
#                     os.remove(f)
#                 except OSError as e:
#                     print("Error: %s : %s" % (f, e.strerror))
                    
# print("There are {} GRBs having no healpix map.".format(len(name_set)))

glg_healpix_all_bn190613449_v00.fit [==============================] 100.00%
glg_skymap_all_bn190613449_v00.png [==============================] 100.00%
glg_loclist_all_bn190613449_v00.txt [==============================] 100.00%
glg_locprob_all_bn190613449_v00.fit [==============================] 100.00%
glg_locplot_all_bn190613449_v00.png [==============================] 100.00%
glg_healpix_all_bn190613172_v00.fit [==============================] 100.00%
glg_skymap_all_bn190613172_v00.png [==============================] 100.00%
glg_loclist_all_bn190613172_v00.txt [==============================] 100.00%
glg_locprob_all_bn190613172_v00.fit [==============================] 100.00%
glg_locplot_all_bn190613172_v00.png [==============================] 100.00%
glg_healpix_all_bn190612165_v00.fit [==============================] 100.00%
glg_skymap_all_bn190612165_v00.png [==============================] 100.00%
glg_loclist_all_bn190612165_v00.txt [==============================] 100.00%
gl

glg_locprob_all_bn190411579_v02.fit [==============================] 100.00%
glg_locplot_all_bn190411579_v02.png [==============================] 100.00%
glg_healpix_all_bn190411407_v00.fit [==============================] 100.00%
glg_skymap_all_bn190411407_v00.png [==============================] 100.00%
glg_loclist_all_bn190411407_v00.txt [==============================] 100.00%
glg_locprob_all_bn190411407_v00.fit [==============================] 100.00%
glg_locplot_all_bn190411407_v00.png [==============================] 100.00%
glg_healpix_all_bn190409901_v00.fit [==============================] 100.00%
glg_skymap_all_bn190409901_v00.png [==============================] 100.00%
glg_locplot_all_bn190409901_v00.png [==============================] 100.00%
glg_healpix_all_bn190407788_v00.fit [==============================] 100.00%
glg_skymap_all_bn190407788_v00.png [==============================] 100.00%
glg_loclist_all_bn190407788_v00.txt [==============================] 100.00%
gl

glg_locplot_all_bn190212482_v00.png [==============================] 100.00%
glg_healpix_all_bn190205938_v01.fit [==============================] 100.00%
glg_skymap_all_bn190205938_v01.png [==============================] 100.00%
glg_loclist_all_bn190205938_v01.txt [==============================] 100.00%
glg_locprob_all_bn190205938_v01.fit [==============================] 100.00%
glg_locplot_all_bn190205938_v01.png [==============================] 100.00%
glg_healpix_all_bn190204627_v00.fit [==============================] 100.00%
glg_skymap_all_bn190204627_v00.png [==============================] 100.00%
glg_loclist_all_bn190204627_v00.txt [==============================] 100.00%
glg_locprob_all_bn190204627_v00.fit [==============================] 100.00%
glg_locplot_all_bn190204627_v00.png [==============================] 100.00%
glg_healpix_all_bn190202634_v00.fit [==============================] 100.00%
glg_skymap_all_bn190202634_v00.png [==============================] 100.00%
gl

glg_locplot_all_bn181122381_v01.png [==============================] 100.00%
glg_healpix_all_bn181121401_v01.fit [==============================] 100.00%
glg_skymap_all_bn181121401_v01.png [==============================] 100.00%
glg_loclist_all_bn181121401_v01.txt [==============================] 100.00%
glg_locprob_all_bn181121401_v01.fit [==============================] 100.00%
glg_locplot_all_bn181121401_v01.png [==============================] 100.00%
glg_healpix_all_bn181121307_v00.fit [==============================] 100.00%
glg_skymap_all_bn181121307_v00.png [==============================] 100.00%
glg_loclist_all_bn181121307_v00.txt [==============================] 100.00%
glg_locprob_all_bn181121307_v00.fit [==============================] 100.00%
glg_locplot_all_bn181121307_v00.png [==============================] 100.00%
glg_healpix_all_bn181120265_v00.fit [==============================] 100.00%
glg_skymap_all_bn181120265_v00.png [==============================] 100.00%
gl

glg_locprob_all_bn180822562_v00.fit [==============================] 100.00%
glg_locplot_all_bn180822562_v00.png [==============================] 100.00%
glg_healpix_all_bn180822423_v00.fit [==============================] 100.00%
glg_skymap_all_bn180822423_v00.png [==============================] 100.00%
glg_loclist_all_bn180822423_v00.txt [==============================] 100.00%
glg_locprob_all_bn180822423_v00.fit [==============================] 100.00%
glg_locplot_all_bn180822423_v00.png [==============================] 100.00%
glg_healpix_all_bn180821653_v01.fit [==============================] 100.00%
glg_skymap_all_bn180821653_v01.png [==============================] 100.00%
glg_loclist_all_bn180821653_v01.txt [==============================] 100.00%
glg_locprob_all_bn180821653_v01.fit [==============================] 100.00%
glg_locplot_all_bn180821653_v01.png [==============================] 100.00%
glg_healpix_all_bn180818179_v00.fit [==============================] 100.00%
g

glg_skymap_all_bn180715755_v00.png [==============================] 100.00%
glg_loclist_all_bn180715755_v00.txt [==============================] 100.00%
glg_locprob_all_bn180715755_v00.fit [==============================] 100.00%
glg_locplot_all_bn180715755_v00.png [==============================] 100.00%
glg_healpix_all_bn180712651_v07.fit [==============================] 100.00%
glg_skymap_all_bn180712651_v07.png [==============================] 100.00%
glg_loclist_all_bn180712651_v07.txt [==============================] 100.00%
glg_locprob_all_bn180712651_v07.fit [==============================] 100.00%
glg_locplot_all_bn180712651_v07.png [==============================] 100.00%
glg_healpix_all_bn180712358_v00.fit [==============================] 100.00%
glg_skymap_all_bn180712358_v00.png [==============================] 100.00%
glg_locplot_all_bn180712358_v00.png [==============================] 100.00%
glg_healpix_all_bn180710062_v00.fit [==============================] 100.00%
gl

glg_locprob_all_bn180610791_v00.fit [==============================] 100.00%
glg_locplot_all_bn180610791_v00.png [==============================] 100.00%
glg_healpix_all_bn180610568_v00.fit [==============================] 100.00%
glg_skymap_all_bn180610568_v00.png [==============================] 100.00%
glg_loclist_all_bn180610568_v00.txt [==============================] 100.00%
glg_locprob_all_bn180610568_v00.fit [==============================] 100.00%
glg_locplot_all_bn180610568_v00.png [==============================] 100.00%
glg_healpix_all_bn180610377_v00.fit [==============================] 100.00%
glg_skymap_all_bn180610377_v00.png [==============================] 100.00%
glg_loclist_all_bn180610377_v00.txt [==============================] 100.00%
glg_locprob_all_bn180610377_v00.fit [==============================] 100.00%
glg_locplot_all_bn180610377_v00.png [==============================] 100.00%
glg_healpix_all_bn180606730_v01.fit [==============================] 100.00%
g

glg_locprob_all_bn180405169_v00.fit [==============================] 100.00%
glg_locplot_all_bn180405169_v00.png [==============================] 100.00%
glg_healpix_all_bn180404848_v00.fit [==============================] 100.00%
glg_skymap_all_bn180404848_v00.png [==============================] 100.00%
glg_loclist_all_bn180404848_v00.txt [==============================] 100.00%
glg_locprob_all_bn180404848_v00.fit [==============================] 100.00%
glg_locplot_all_bn180404848_v00.png [==============================] 100.00%
glg_healpix_all_bn180404091_v00.fit [==============================] 100.00%
glg_skymap_all_bn180404091_v00.png [==============================] 100.00%
glg_loclist_all_bn180404091_v00.txt [==============================] 100.00%
glg_locprob_all_bn180404091_v00.fit [==============================] 100.00%
glg_locplot_all_bn180404091_v00.png [==============================] 100.00%
glg_healpix_all_bn180403565_v00.fit [==============================] 100.00%
g

glg_healpix_all_bn180119837_v00.fit [==============================] 100.00%
glg_skymap_all_bn180119837_v00.png [==============================] 100.00%
glg_loclist_all_bn180119837_v00.txt [==============================] 100.00%
glg_locprob_all_bn180119837_v00.fit [==============================] 100.00%
glg_locplot_all_bn180119837_v00.png [==============================] 100.00%
glg_healpix_all_bn180116678_v00.fit [==============================] 100.00%
glg_skymap_all_bn180116678_v00.png [==============================] 100.00%
glg_loclist_all_bn180116678_v00.txt [==============================] 100.00%
glg_locprob_all_bn180116678_v00.fit [==============================] 100.00%
glg_locplot_all_bn180116678_v00.png [==============================] 100.00%
glg_healpix_all_bn180116026_v00.fit [==============================] 100.00%
glg_skymap_all_bn180116026_v00.png [==============================] 100.00%
glg_loclist_all_bn180116026_v00.txt [==============================] 100.00%
gl

It looks like fermi FTP (`https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/`) does not have GRB healpix files before 180112

Evetually, I got outdated but complete healpix files from Liz \
They are here: `/data/user/cjchen/2020-2021/Spring2021/greco_grb/data/gbm/healpix_liz`
Those files are originally located at: `/data/user/efried/IceCube/gbm_2008_2019/gbm_healpix_2008_july2019/healpix` \
or `/cvmfs/icecube.opensciencegrid.org/users/efried/GRB_coincidence/binomial_test/healpix/`

In [57]:
ftp_folder = "/home/cjchen/2020-2021/Spring2021/greco_grb/data/gbm/healpix_ftp"
liz_folder = "/home/cjchen/2020-2021/Spring2021/greco_grb/data/gbm/healpix_liz"
ftp_files = glob.glob('{}/*_healpix_*.fit'.format(ftp_folder))
liz_files = glob.glob('{}/*_healpix_*.fit'.format(liz_folder))


In [89]:
out_folder = "/home/cjchen/2020-2021/Spring2021/greco_grb/data/gbm/healpix"

In [83]:
ftp_map = dict()
liz_map = dict()
for file in ftp_files:
    filename = os.path.basename(file)
    ftp_map[filename[18:27]] = file
for file in liz_files:
    filename = os.path.basename(file)
    liz_map[filename[18:27]] = file

In [90]:
no_gbm_list = []
name_set = set(df.name)
for name in grbweb.grb_name_GBM:
    if name not in name_set:
        continue
    """
    # We got len(no_gbm_list) = 33 as well
    if any(name[3:] in filename for filename in ftp_files):
        pass
    elif any(name[3:] in filename for filename in liz_files):
        pass
    else:
        no_gbm_list.append(name)
    """
    
    if name[3:] in ftp_map:
        try:
            os.system('cp {} {}'.format(ftp_map[name[3:]], out_folder+"/"))  
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))
    elif name[3:] in liz_map:
        try:
            os.system('cp {} {}'.format(liz_map[name[3:]], out_folder+"/"))  
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))
    else:
        no_gbm_list.append(name)

In [91]:
len(no_gbm_list)

33

In [93]:
print(no_gbm_list)

['GRB180205184', 'GRB180128252', 'GRB171119992', 'GRB170416583', 'GRB170203486', 'GRB170113420', 'GRB161218222', 'GRB161121186', 'GRB161020024', 'GRB160314929', 'GRB160220868', 'GRB151218857', 'GRB150917148', 'GRB150618674', 'GRB150513856', 'GRB150411026', 'GRB150326542', 'GRB150325696', 'GRB140912664', 'GRB140825980', 'GRB140711691', 'GRB140628704', 'GRB140501139', 'GRB140219319', 'GRB140115899', 'GRB140109877', 'GRB131029990', 'GRB130925164', 'GRB130909817', 'GRB130908677', 'GRB120817168', 'GRB120624309', 'GRB120513531']


In [1]:
import pickle
no_gbm_list = ['GRB180205184', 'GRB180128252', 'GRB171119992', 'GRB170416583', 'GRB170203486', 'GRB170113420', 'GRB161218222', 'GRB161121186', 'GRB161020024', 'GRB160314929', 'GRB160220868', 'GRB151218857', 'GRB150917148', 'GRB150618674', 'GRB150513856', 'GRB150411026', 'GRB150326542', 'GRB150325696', 'GRB140912664', 'GRB140825980', 'GRB140711691', 'GRB140628704', 'GRB140501139', 'GRB140219319', 'GRB140115899', 'GRB140109877', 'GRB131029990', 'GRB130925164', 'GRB130909817', 'GRB130908677', 'GRB120817168', 'GRB120624309', 'GRB120513531']


In [2]:
with open("/home/cjchen/2020-2021/Spring2021/greco_grb/data/gbm/no_healpix_gbm_grb_list.txt", "wb") as fp:   #Pickling
    pickle.dump(no_gbm_list, fp)
    

In [3]:
with open("/home/cjchen/2020-2021/Spring2021/greco_grb/data/gbm/no_healpix_gbm_grb_list.txt", "rb") as fp:   # Unpickling
    b = pickle.load(fp)

In [4]:
b

['GRB180205184',
 'GRB180128252',
 'GRB171119992',
 'GRB170416583',
 'GRB170203486',
 'GRB170113420',
 'GRB161218222',
 'GRB161121186',
 'GRB161020024',
 'GRB160314929',
 'GRB160220868',
 'GRB151218857',
 'GRB150917148',
 'GRB150618674',
 'GRB150513856',
 'GRB150411026',
 'GRB150326542',
 'GRB150325696',
 'GRB140912664',
 'GRB140825980',
 'GRB140711691',
 'GRB140628704',
 'GRB140501139',
 'GRB140219319',
 'GRB140115899',
 'GRB140109877',
 'GRB131029990',
 'GRB130925164',
 'GRB130909817',
 'GRB130908677',
 'GRB120817168',
 'GRB120624309',
 'GRB120513531']